In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [3]:
df =  pd.read_csv('/content/drive/MyDrive/Mltiverse/MLtiVerse (1)/test.csv')

In [4]:
## importaant function that will be used during whole cycle

#to display list of all features that contain missing values
def missValueFeatures(dataset):
  features = [features for features in dataset.columns if dataset[features].isnull().sum()>0]
  return features

#To draw a plot that shows the relationship between missing values and present value with respect to target variable
def missValuePlot(dataset, li, target_variable):
  for feature in li:
    data = dataset.copy()
    print(data[feature].isnull().sum())
    data[feature] = np.where(data[feature].isnull(),1,0)
    data.groupby(feature)[target_variable].mean().plot.bar()
    plt.title(feature)
    plt.show()

#seperating independent and dependent feature
def depIndepFeature(dataset, dep_feature):
  X = dataset.drop(dep_feature, axis=1)
  y = pd.DataFrame(dataset[dep_feature])
  return X,y

#for univariate feature imputation
#Here data is all those rows and columns where you want to apply imputation
from sklearn.impute import SimpleImputer
def simpleImputation(data, estimator):
  from sklearn.impute import SimpleImputer
  imputer = SimpleImputer(missing_values=np.nan, strategy=estimator)
  imputer.fit(data)
  data = imputer.transform(data)
  return data

#for multivariate feature imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
def iterativeImputation(data):
  imputer = IterativeImputer(max_iter=10, random_state=0)
  imputer.fit(data)
  data = imputer.transform(data)
  return data

#removing rows containing null values
def nullValuesRemoval(dataset, li):
  df = dataset.dropna(subset = li)
  return df

#list of all numerical features
def numericalFeatures(dataset):
  numerical_features = [features for features in dataset.columns if dataset[features].dtypes != 'O']
  return numerical_features

#list of all discrete numerical features
def discreteNumericalFeatures(dataset, numerical_features):
  discreteNumericalFeatures = [feature for feature in numerical_features if len(dataset[feature].unique())<25 and feature not in ['ID']]
  return discreteNumericalFeatures

#list of all continuous features
def continuousNumericalFeatures(dataset, numerical_features, discrete_numerical_features):
  continuous_numerical_features = [feature for feature in numerical_features if feature not in discrete_numerical_features + ['ID']]
  return continuous_numerical_features
#list of all categorical features
def categoricalFeatures(dataset, numerical_features, temporal_features):
  categorical_features = [features for features in dataset.columns if dataset[features].dtypes == 'O' and features not in numerical_features + temporal_features]
  return categorical_features

#Relationship between discrete numerical features and target variable
def discreteTargetRelation(dataset, discrete_numerical_features, target_variable):
  for feature in discrete_numerical_features:
    data = dataset.copy()
    data.groupby(feature)[target_variable].mean().plot.bar()
    plt.xlabel(feature)
    plt.ylabel(target_variable)
    plt.title(feature)
    plt.show()

#For plotting Pdf curve for continuous numericl feature
def pdf(dataset, continuous_numerical_features):
  for feature in continuous_numerical_features:
    data = dataset.copy()
    sns.distplot(data[feature])
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.title(feature)
    plt.show()

#Relationship between continuous features and target variable
def continuousTargetRelation(dataset, continuous_numerical_features, target_variable):
  for feature in continuous_numerical_features:
    data = dataset.copy()
    if 0 in data[feature].unique():
      pass
    else:
      data[feature] = np.log(data[feature])
      data[target_variable] = np.log(data[target_variable])
      plt.scatter(data[feature], data[target_variable])
      plt.xlabel(feature)
      plt.ylabel(target_variable)
      plt.title(feature)
      plt.show()

#For plotting outliers
def plotOutliers(dataset, continuous_numerical_features):
  for feature in continuous_numerical_features:
    data = dataset.copy()
    if 0 in data[feature].unique():
      pass
    else:
      data[feature] = np.log(data[feature])
      data.boxplot(column = feature)
      plt.ylabel(feature)
      plt.title(feature)
      plt.show()

# Defining a function to calculate the number of outliers using z-score method
def count_outliers_zscore(data, threshold=3):
    z_scores = np.abs((data - data.mean()) / data.std())
    num_outliers = (z_scores > threshold).sum()
    return num_outliers

#For number of outliers using DBSCAN algo.
from sklearn.cluster import DBSCAN
def find_outliers_dbscan(data, epsilon, min_samples):
    dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)
    dbscan.fit(data)
    labels = dbscan.labels_
    labels_list = list(labels)
    # Count the number of outliers for each feature
    num_outliers = labels_list.count(-1)
    return num_outliers

#For number of outliers using ECOD PyOD algo.
!pip install pyod
from pyod.models.ecod import ECOD
def detect_outliers_ecod(data, threshold=95):
    # Convert the data to a NumPy array
    X = np.array(data)

    # Create an ECOD model and fit it to the data
    model = ECOD()
    model.fit(X)

    # Obtain the outlier scores for each data point
    scores = model.decision_scores_

    # Define the threshold based on the given percentile
    threshold_value = np.percentile(scores, threshold)

    # Detect outliers for each feature
    outliers = []
    for i in range(X.shape[1]):
        feature_scores = scores[:, i]
        feature_outliers = X[np.where(feature_scores > threshold_value)]
        outliers.append(feature_outliers)

    # Return the outliers for each feature
    return outliers


#Log normal transformatioon to remove outliers
def log_normal_transform(dataset):
    transformed_data = dataset.copy()
    non_zero_mask = transformed_data != 0
    transformed_data[non_zero_mask] = np.log(transformed_data[non_zero_mask])
    return transformed_data

#Flooring and capping for removal of outliers
def quantile_floor_cap(dataset, column, lower_quantile=0.05, upper_quantile=0.95):
    lower_bound = dataset[column].quantile(lower_quantile)
    upper_bound = dataset[column].quantile(upper_quantile)
    dataset[column] = dataset[column].apply(lambda x: lower_bound if x < lower_bound else upper_bound if x > upper_bound else x)
    return dataset

#converting the rare categorical values to rare_var
def rareVarConversion(dataset, target_variable, features):
  for feature in features:
    temp = dataset.groupby(feature)[target_variable].count()/len(dataset)
    temp_df = temp[temp>0.0005].index
    dataset[feature] = np.where(dataset[feature].isin(temp_df),dataset[feature],"Rare_var")
    return dataset

#Remove all outliers from a dataset
from scipy import stats
def remove_outliers(data, threshold=3):
    z_scores = np.abs(stats.zscore(data))
    outliers_mask = np.any(z_scores > threshold, axis=1)
    cleaned_data = data[~outliers_mask]
    return cleaned_data

#function for building ann model
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
def build_model_using_sequential(hidden_units1, hidden_units2, hidden_units3, learning_rate):
  model = Sequential([
    Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='linear')
  ])
  return model

def remove_outlier(df_in, col_name):
  q1 = df_in[col_name].quantile(0.25)
  q3 = df_in[col_name].quantile(0.75)
  iqr = q3-q1 #Interquartile range
  fence_low  = q1-1.5*iqr
  fence_high = q3+1.5*iqr
  df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
  return df_out

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.1.0-py3-none-any.whl size=185331 sha256=bc9ce6826584c7c2fb2e53c8aceb4074a61b9eba54b78f8e9265f7fdb3a8f5db
  Stored in directory: /root/.cache/pip/wheels/36/8e/e2/e932956b10b843eb6be9eefa70b5c1bee7b561be14c423b136
Successfully built pyod


In [5]:
df['Wage'] = df['Wage'].apply(lambda x:  (float(x[1:-1]) * 1000) if 'K' in x else (float(x[1:]) ))
from pandas.core.dtypes.dtypes import dtypes
data = df.copy()
data1 = data["Release Clause"]
data_numeric = []

for value in data1:
    if isinstance(value, str):
        numeric_value = float(value[1:-1]) * 1e6  if value[-1] == 'M' else float(value[1:-1]) * 1e3
        data_numeric.append(numeric_value)
    else:
        data_numeric.append(np.nan)
series = pd.Series(data_numeric)
median = series.median()
df["Release Clause"] = df["Release Clause"].fillna(median)
df["Release Clause"] = df["Release Clause"].apply(lambda x: x if type(x) == float else (float(x[1:-1]) * 1e3) if 'K' else (float(x[1:-1]) * 1e6) if 'M' in x else x)

In [6]:
data = df.copy()
temporal_features = ["Joined" , "Contract Valid Until"]
data['Joined'] = pd.to_datetime(data['Joined'])
data['Contract Valid Until'] = pd.to_datetime(data['Contract Valid Until'])

# # Calculate the number of days
df['Days_of_contract'] = (data['Contract Valid Until'] - data['Joined']).dt.days
df.drop(['Joined', 'Contract Valid Until'], axis=1, inplace=True)

In [7]:
df['Weight'] = df['Weight'].apply(lambda x: int(x[0:2])*2.205)
df["Preferred Foot"] = df["Preferred Foot"].apply(lambda x : 1 if "Right" in x else 0)
df["Height"] = df["Height"].apply(lambda x : int(int(x[0:3])*0.394))

In [8]:
df

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Loaned From,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness,year,Days_of_contract
0,212198,Bruno Fernandes,26,https://cdn.sofifa.com/players/212/198/22_60.png,Portugal,https://cdn.sofifa.com/flags/pt.png,88,89,Manchester United,https://cdn.sofifa.com/teams/11/30.png,250000.0,2341,1,3.0,3.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos18"">CAM",18.0,NaN,70,152.145,87.0,83.0,64.0,91.0,87.0,83.0,87.0,87.0,88.0,87.0,77.0,73.0,80.0,91.0,79.0,89.0,73.0,91.0,70.0,89.0,78.0,66.0,87.0,90.0,91.0,87.0,NaN,73.0,65.0,12.0,14.0,15.0,8.0,14.0,CAM,88.0,206900.0,72.0,2022,1798.0
1,209658,L. Goretzka,26,https://cdn.sofifa.com/players/209/658/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.com/teams/21/30.png,140000.0,2314,1,4.0,4.0,3.0,High/ Medium,Unique,Yes,"<span class=""pos pos11"">LDM",8.0,NaN,74,180.810,75.0,82.0,86.0,86.0,69.0,84.0,76.0,75.0,84.0,87.0,78.0,83.0,76.0,88.0,71.0,85.0,79.0,88.0,88.0,86.0,81.0,86.0,85.0,84.0,60.0,82.0,NaN,85.0,77.0,13.0,8.0,15.0,11.0,9.0,CM,87.0,160400.0,74.0,2022,2741.0
2,176580,L. Suárez,34,https://cdn.sofifa.com/players/176/580/22_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,88,88,Atlético de Madrid,https://cdn.sofifa.com/teams/240/30.png,135000.0,2307,1,5.0,4.0,3.0,High/ Medium,Unique,Yes,"<span class=""pos pos24"">RS",9.0,NaN,71,183.015,80.0,93.0,84.0,83.0,90.0,83.0,86.0,82.0,77.0,86.0,76.0,69.0,75.0,92.0,78.0,89.0,69.0,78.0,85.0,88.0,87.0,41.0,91.0,84.0,83.0,87.0,NaN,45.0,38.0,27.0,25.0,31.0,33.0,37.0,ST,88.0,91200.0,42.0,2022,463.0
3,192985,K. De Bruyne,30,https://cdn.sofifa.com/players/192/985/22_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,91,91,Manchester City,https://cdn.sofifa.com/teams/10/30.png,350000.0,2304,1,4.0,5.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos13"">RCM",17.0,NaN,71,154.350,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93.0,91.0,76.0,76.0,79.0,91.0,78.0,91.0,63.0,89.0,74.0,91.0,76.0,66.0,88.0,94.0,83.0,89.0,NaN,65.0,53.0,15.0,13.0,5.0,10.0,13.0,CM,91.0,232200.0,68.0,2022,3412.0
4,224334,M. Acuña,29,https://cdn.sofifa.com/players/224/334/22_60.png,Argentina,https://cdn.sofifa.com/flags/ar.png,84,84,Sevilla FC,https://cdn.sofifa.com/teams/481/30.png,45000.0,2292,0,2.0,3.0,4.0,High/ High,Stocky (170-185),No,"<span class=""pos pos7"">LB",19.0,NaN,67,152.145,87.0,66.0,58.0,82.0,68.0,87.0,88.0,75.0,78.0,88.0,77.0,76.0,83.0,83.0,90.0,82.0,63.0,90.0,80.0,81.0,84.0,79.0,81.0,82.0,76.0,87.0,NaN,84.0,82.0,8.0,14.0,13.0,13.0,14.0,LB,84.0,77700.0,80.0,2022,1204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16705,240558,18 L. Clayton,17,https://cdn.sofifa.com/players/240/558/18_60.png,England,https://cdn.sofifa.com/flags/gb-eng.png,53,70,Cheltenham Town,https://cdn.sofifa.com/teams/1936/30.png,1000.0,782,1,1.0,2.0,1.0,Medium/ Medium,Normal (185+),No,"<span class=""pos pos29"">RES",31.0,NaN,74,176.400,12.0,5.0,13.0,18.0,7.0,12.0,11.0,13.0,19.0,10.0,15.0,21.0,34.0,30.0,31.0,16.0,51.0,26.0,57.0,6.0,16.0,9.0,5.0,29.0,18.0,37.0,5.0,11.0,12.0,55.0,54.0,52.0,50.0,59.0,GK,52.0,238000.0,NaN,2022,914.0
16706,262846,�. Dobre,20,https://cdn.sofifa.com/players/262/846/22_60.png,Romania,https://cdn.sofifa.com/flags/ro.png,53,63,FC Academica Clinceni,https

In [9]:
features_with_missing_values = missValueFeatures(df)
features_with_missing_values

['Club',
 'Body Type',
 'Real Face',
 'Position',
 'Jersey Number',
 'Loaned From',
 'Volleys',
 'Curve',
 'Agility',
 'Balance',
 'Jumping',
 'Interceptions',
 'Positioning',
 'Vision',
 'Composure',
 'Marking',
 'SlidingTackle',
 'DefensiveAwareness',
 'Days_of_contract']

In [10]:
# features = ['Club', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Volleys', 'Curve', 'Agility', 'Balance', 'Jumping', 'Interceptions', 'Positioning', 'Vision', 'SlidingTackle']
# for feature in features:
#   dataset = df[df[feature] == 0]

In [11]:
features = ['Club', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Volleys', 'Curve', 'Agility', 'Balance', 'Jumping', 'Interceptions', 'Positioning', 'Vision', 'SlidingTackle']

filtered_df = df[df[features].isnull()]
df_new = nullValuesRemoval(df, features)

In [12]:
test_data_photo = df_new["Photo"]

In [13]:
test_data_photo.to_csv("/content/drive/MyDrive/Mltiverse/MLtiVerse (1)/test_data_photo.csv", index = False)

In [14]:
filtered_df

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Loaned From,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness,year,Days_of_contract
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16707,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16708,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_new

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Loaned From,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness,year,Days_of_contract
0,212198,Bruno Fernandes,26,https://cdn.sofifa.com/players/212/198/22_60.png,Portugal,https://cdn.sofifa.com/flags/pt.png,88,89,Manchester United,https://cdn.sofifa.com/teams/11/30.png,250000.0,2341,1,3.0,3.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos18"">CAM",18.0,NaN,70,152.145,87.0,83.0,64.0,91.0,87.0,83.0,87.0,87.0,88.0,87.0,77.0,73.0,80.0,91.0,79.0,89.0,73.0,91.0,70.0,89.0,78.0,66.0,87.0,90.0,91.0,87.0,NaN,73.0,65.0,12.0,14.0,15.0,8.0,14.0,CAM,88.0,206900.0,72.0,2022,1798.0
1,209658,L. Goretzka,26,https://cdn.sofifa.com/players/209/658/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.com/teams/21/30.png,140000.0,2314,1,4.0,4.0,3.0,High/ Medium,Unique,Yes,"<span class=""pos pos11"">LDM",8.0,NaN,74,180.810,75.0,82.0,86.0,86.0,69.0,84.0,76.0,75.0,84.0,87.0,78.0,83.0,76.0,88.0,71.0,85.0,79.0,88.0,88.0,86.0,81.0,86.0,85.0,84.0,60.0,82.0,NaN,85.0,77.0,13.0,8.0,15.0,11.0,9.0,CM,87.0,160400.0,74.0,2022,2741.0
2,176580,L. Suárez,34,https://cdn.sofifa.com/players/176/580/22_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,88,88,Atlético de Madrid,https://cdn.sofifa.com/teams/240/30.png,135000.0,2307,1,5.0,4.0,3.0,High/ Medium,Unique,Yes,"<span class=""pos pos24"">RS",9.0,NaN,71,183.015,80.0,93.0,84.0,83.0,90.0,83.0,86.0,82.0,77.0,86.0,76.0,69.0,75.0,92.0,78.0,89.0,69.0,78.0,85.0,88.0,87.0,41.0,91.0,84.0,83.0,87.0,NaN,45.0,38.0,27.0,25.0,31.0,33.0,37.0,ST,88.0,91200.0,42.0,2022,463.0
3,192985,K. De Bruyne,30,https://cdn.sofifa.com/players/192/985/22_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,91,91,Manchester City,https://cdn.sofifa.com/teams/10/30.png,350000.0,2304,1,4.0,5.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos13"">RCM",17.0,NaN,71,154.350,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93.0,91.0,76.0,76.0,79.0,91.0,78.0,91.0,63.0,89.0,74.0,91.0,76.0,66.0,88.0,94.0,83.0,89.0,NaN,65.0,53.0,15.0,13.0,5.0,10.0,13.0,CM,91.0,232200.0,68.0,2022,3412.0
4,224334,M. Acuña,29,https://cdn.sofifa.com/players/224/334/22_60.png,Argentina,https://cdn.sofifa.com/flags/ar.png,84,84,Sevilla FC,https://cdn.sofifa.com/teams/481/30.png,45000.0,2292,0,2.0,3.0,4.0,High/ High,Stocky (170-185),No,"<span class=""pos pos7"">LB",19.0,NaN,67,152.145,87.0,66.0,58.0,82.0,68.0,87.0,88.0,75.0,78.0,88.0,77.0,76.0,83.0,83.0,90.0,82.0,63.0,90.0,80.0,81.0,84.0,79.0,81.0,82.0,76.0,87.0,NaN,84.0,82.0,8.0,14.0,13.0,13.0,14.0,LB,84.0,77700.0,80.0,2022,1204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16704,251433,B. Voll,20,https://cdn.sofifa.com/players/251/433/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,58,69,F.C. Hansa Rostock,https://cdn.sofifa.com/teams/27/30.png,950.0,802,1,1.0,2.0,1.0,Medium/ Medium,Normal (185+),No,"<span class=""pos pos29"">RES",30.0,NaN,76,187.425,8.0,9.0,11.0,23.0,7.0,11.0,11.0,11.0,17.0,14.0,24.0,25.0,27.0,42.0,20.0,42.0,22.0,21.0,67.0,8.0,21.0,4.0,5.0,27.0,7.0,29.0,NaN,12.0,10.0,59.0,60.0,56.0,55.0,61.0,GK,58.0,656000.0,5.0,2022,915.0
16705,240558,18 L. Clayton,17,https://cdn.sofifa.com/players/240/558/18_60.png,England,https://cdn.sofifa.com/flags/gb-eng.png,53,70,Cheltenham Town,https://cdn.so

In [16]:
features_with_missing_values = missValueFeatures(df_new)
features_with_missing_values

['Loaned From',
 'Composure',
 'Marking',
 'DefensiveAwareness',
 'Days_of_contract']

In [17]:
features = ['Club', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Volleys', 'Curve', 'Agility', 'Balance', 'Jumping', 'Interceptions', 'Positioning', 'Vision', 'SlidingTackle']
filtered_df = nullValuesRemoval(filtered_df, features)

In [18]:
photo = df_new["Photo"]
df_new.drop(['ID', 'Name', 'Photo', 'Flag', 'Club Logo', 'Loaned From', 'Nationality', 'Club', 'Position', 'Best Position'], axis=1, inplace=True)

<ipython-input-18-beebe09c22ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.drop(['ID', 'Name', 'Photo', 'Flag', 'Club Logo', 'Loaned From', 'Nationality', 'Club', 'Position', 'Best Position'], axis=1, inplace=True)


In [19]:
numerical_features = numericalFeatures(df_new)

In [20]:
categorical_features = categoricalFeatures(df_new, numerical_features, temporal_features)

In [21]:
multivariate_imputed_X = iterativeImputation(df_new[numerical_features])

In [22]:
multivariate_imputed_X = pd.DataFrame(multivariate_imputed_X)

In [23]:
categorical_df = df_new[categorical_features]
multivariate_imputed_X.reset_index(drop=True, inplace=True)
categorical_df.reset_index(drop=True, inplace=True)
df_multivariate_imputed = pd.concat([multivariate_imputed_X, categorical_df], axis=1)

In [24]:
df_multivariate_imputed.isnull().sum().sum()

0

In [25]:
!pip install category_encoders
import category_encoders as ce
encoder= ce.OrdinalEncoder(cols=['Work Rate'],return_df=True,
                           mapping=[{'col':'Work Rate',
'mapping':{'NA/ NA':0,'Low/ Low':1,'Low/ Medium':2,'Medium/ Medium':3, 'Low/ High':4, 'Medium/ Low':5, 'High/ Low':6, 'Medium/ High':7, 'High/ Medium':8, 'High/ High':9}}])
df_multivariate_imputed['Work_Rate_transformed'] = encoder.fit_transform(df_multivariate_imputed['Work Rate'])
df_multivariate_imputed.drop("Work Rate", axis = 1, inplace = True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.8 MB/s eta 0:00:00


In [26]:
encoder= ce.OrdinalEncoder(cols=['Real Face'],return_df=True,
                           mapping=[{'col':'Real Face',
'mapping':{'No':0, 'Yes':1}}])
df_multivariate_imputed['Real_Face_transformed'] = encoder.fit_transform(df_multivariate_imputed['Real Face'])
df_multivariate_imputed.drop("Real Face", axis = 1, inplace = True)

In [27]:
df_multivariate_imputed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Body Type,Work_Rate_transformed,Real_Face_transformed
0,26.0,88.0,89.0,250000.0,2341.0,1.0,3.0,3.0,4.0,18.0,70.0,152.145,87.0,83.0,64.0,91.0,87.0,83.0,87.0,87.0,88.0,87.0,77.0,73.0,80.0,91.0,79.0,89.0,73.0,91.0,70.0,89.0,78.0,66.0,87.0,90.0,91.0,87.0,66.363477,73.0,65.0,12.0,14.0,15.0,8.0,14.0,88.0,206900.0,72.000000,2022.0,1798.0,Unique,9.0,1
1,26.0,87.0,88.0,140000.0,2314.0,1.0,4.0,4.0,3.0,8.0,74.0,180.810,75.0,82.0,86.0,86.0,69.0,84.0,76.0,75.0,84.0,87.0,78.0,83.0,76.0,88.0,71.0,85.0,79.0,88.0,88.0,86.0,81.0,86.0,85.0,84.0,60.0,82.0,72.193359,85.0,77.0,13.0,8.0,15.0,11.0,9.0,87.0,160400.0,74.000000,2022.0,2741.0,Unique,8.0,1
2,34.0,88.0,88.0,135000.0,2307.0,1.0,5.0,4.0,3.0,9.0,71.0,183.015,80.0,93.0,84.0,83.0,90.0,83.0,86.0,82.0,77.0,86.0,76.0,69.0,75.0,92.0,78.0,89.0,69.0,78.0,85.0,88.0,87.0,41.0,91.0,84.0,83.0,87.0,41.260758,45.0,38.0,27.0,25.0,31.0,33.0,37.0,88.0,91200.0,42.000000,2022.0,463.0,Unique,8.0,1
3,30.0,91.0,91.0,350000.0,2304.0,1.0,4.0,5.0,4.0,17.0,71.0,154.350,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93.0,91.0,76.0,76.0,79.0,91.0,78.0,91.0,63.0,89.0,74.0,91.0,76.0,66.0,88.0,94.0,83.0,89.0,61.788810,65.0,53.0,15.0,13.0,5.0,10.0,13.0,91.0,232200.0,68.000000,2022.0,3412.0,Unique,9.0,1
4,29.0,84.0,84.0,45000.0,2292.0,0.0,2.0,3.0,4.0,19.0,67.0,152.145,87.0,66.0,58.0,82.0,68.0,87.0,88.0,75.0,78.0,88.0,77.0,76.0,83.0,83.0,90.0,82.0,63.0,90.0,80.0,81.0,84.0,79.0,81.0,82.0,76.0,87.0,74.381081,84.0,82.0,8.0,14.0,13.0,13.0,14.0,84.0,77700.0,80.000000,2022.0,1204.0,Stocky (170-185),9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16411,20.0,58.0,69.0,950.0,802.0,1.0,1.0,2.0,1.0,30.0,76.0,187.425,8.0,9.0,11.0,23.0,7.0,11.0,11.0,11.0,17.0,14.0,24.0,25.0,27.0,42.0,20.0,42.0,22.0,21.0,67.0,8.0,21.0,4.0,5.0,27.0,7.0,29.0,10.630345,12.0,10.0,59.0,60.0,56.0,55.0,61.0,58.0,656000.0,5.000000,2022.0,915.0,Normal (185+),3.0,0
16412,17.0,53.0,70.0,1000.0,782.0,1.0,1.0,2.0,1.0,31.0,74.0,176.400,12.0,5.0,13.0,18.0,7.0,12.0,11.0,13.0,19.0,10.0,15.0,21.0,34.0,30.0,31.0,16.0,51.0,26.0,57.0,6.0,16.0,9.0,5.0,29.0,18.0,37.0,5.000000,11.0,12.0,55.0,54.0,52.0,50.0,59.0,52.0,238000.0,-2.823051,2022.0,914.0,Normal (185+),3.0,0
16413,20.0,53.0,63.0,550.0,778.0,1.0,1.0,2.0,1.0,1.0,76.0,189.630,13.0,7.0,10.0,17.0,7.0,6.0,11.0,13.0,28.0,17.0,16.0,22.0,30.0,38.0,24.0,40.0,35.0,24.0,32.0,6.0,18.0,13.0,8.0,33.0,12.0,29.0,12.165843,13.0,12.0,57.0,52.0,53.0,48.0,58.0,53.0,279000.0,5.000000,2022.0,2349.0,Normal (185+),3.0,0
16414,19.0,47.0,60.0,700.0,770.0,1.0,1.0,2.0,1.0,31.0,70.0,156.555,12.0,9.0,14.0,15.0,6.0,8.0,12.0,13.0,14.0,12.0,22.0,17.0,36.0,43.0,36.0,34.0,51.0,20.0,51.0,8.0,13.0,11.0,7.0,18.0,15.0,17.0,19.406682,11.0,9.0,49.0,48.0,45.0,38.0,52.0,47.0,223000.0,21.000000,2022.0,-108.0,Lean (170-185),3.0,0


In [28]:
features = ['Body Type']
target_variable = 3
df_multivariate_imputed = rareVarConversion(df_multivariate_imputed, target_variable, features)

In [29]:
df_multivariate_imputed['Body Type'].value_counts()

Normal (170-185)    5630
Normal (185+)       3595
Lean (170-185)      3431
Lean (185+)         1669
Stocky (170-185)     563
Normal (170-)        545
Lean (170-)          376
Stocky (185+)        364
Unique               139
Stocky (170-)        104
Name: Body Type, dtype: int64

In [30]:
encoder = ce.BinaryEncoder(cols=['Body Type'],return_df=True)
df_multivariate_imputed = encoder.fit_transform(df_multivariate_imputed)
df_multivariate_imputed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Body Type_0,Body Type_1,Body Type_2,Body Type_3,Work_Rate_transformed,Real_Face_transformed
0,26.0,88.0,89.0,250000.0,2341.0,1.0,3.0,3.0,4.0,18.0,70.0,152.145,87.0,83.0,64.0,91.0,87.0,83.0,87.0,87.0,88.0,87.0,77.0,73.0,80.0,91.0,79.0,89.0,73.0,91.0,70.0,89.0,78.0,66.0,87.0,90.0,91.0,87.0,66.363477,73.0,65.0,12.0,14.0,15.0,8.0,14.0,88.0,206900.0,72.000000,2022.0,1798.0,0,0,0,1,9.0,1
1,26.0,87.0,88.0,140000.0,2314.0,1.0,4.0,4.0,3.0,8.0,74.0,180.810,75.0,82.0,86.0,86.0,69.0,84.0,76.0,75.0,84.0,87.0,78.0,83.0,76.0,88.0,71.0,85.0,79.0,88.0,88.0,86.0,81.0,86.0,85.0,84.0,60.0,82.0,72.193359,85.0,77.0,13.0,8.0,15.0,11.0,9.0,87.0,160400.0,74.000000,2022.0,2741.0,0,0,0,1,8.0,1
2,34.0,88.0,88.0,135000.0,2307.0,1.0,5.0,4.0,3.0,9.0,71.0,183.015,80.0,93.0,84.0,83.0,90.0,83.0,86.0,82.0,77.0,86.0,76.0,69.0,75.0,92.0,78.0,89.0,69.0,78.0,85.0,88.0,87.0,41.0,91.0,84.0,83.0,87.0,41.260758,45.0,38.0,27.0,25.0,31.0,33.0,37.0,88.0,91200.0,42.000000,2022.0,463.0,0,0,0,1,8.0,1
3,30.0,91.0,91.0,350000.0,2304.0,1.0,4.0,5.0,4.0,17.0,71.0,154.350,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93.0,91.0,76.0,76.0,79.0,91.0,78.0,91.0,63.0,89.0,74.0,91.0,76.0,66.0,88.0,94.0,83.0,89.0,61.788810,65.0,53.0,15.0,13.0,5.0,10.0,13.0,91.0,232200.0,68.000000,2022.0,3412.0,0,0,0,1,9.0,1
4,29.0,84.0,84.0,45000.0,2292.0,0.0,2.0,3.0,4.0,19.0,67.0,152.145,87.0,66.0,58.0,82.0,68.0,87.0,88.0,75.0,78.0,88.0,77.0,76.0,83.0,83.0,90.0,82.0,63.0,90.0,80.0,81.0,84.0,79.0,81.0,82.0,76.0,87.0,74.381081,84.0,82.0,8.0,14.0,13.0,13.0,14.0,84.0,77700.0,80.000000,2022.0,1204.0,0,0,1,0,9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16411,20.0,58.0,69.0,950.0,802.0,1.0,1.0,2.0,1.0,30.0,76.0,187.425,8.0,9.0,11.0,23.0,7.0,11.0,11.0,11.0,17.0,14.0,24.0,25.0,27.0,42.0,20.0,42.0,22.0,21.0,67.0,8.0,21.0,4.0,5.0,27.0,7.0,29.0,10.630345,12.0,10.0,59.0,60.0,56.0,55.0,61.0,58.0,656000.0,5.000000,2022.0,915.0,0,1,1,0,3.0,0
16412,17.0,53.0,70.0,1000.0,782.0,1.0,1.0,2.0,1.0,31.0,74.0,176.400,12.0,5.0,13.0,18.0,7.0,12.0,11.0,13.0,19.0,10.0,15.0,21.0,34.0,30.0,31.0,16.0,51.0,26.0,57.0,6.0,16.0,9.0,5.0,29.0,18.0,37.0,5.000000,11.0,12.0,55.0,54.0,52.0,50.0,59.0,52.0,238000.0,-2.823051,2022.0,914.0,0,1,1,0,3.0,0
16413,20.0,53.0,63.0,550.0,778.0,1.0,1.0,2.0,1.0,1.0,76.0,189.630,13.0,7.0,10.0,17.0,7.0,6.0,11.0,13.0,28.0,17.0,16.0,22.0,30.0,38.0,24.0,40.0,35.0,24.0,32.0,6.0,18.0,13.0,8.0,33.0,12.0,29.0,12.165843,13.0,12.0,57.0,52.0,53.0,48.0,58.0,53.0,279000.0,5.000000,2022.0,2349.0,0,1,1,0,3.0,0
16414,19.0,47.0,60.0,700.0,770.0,1.0,1.0,2.0,1.0,31.0,70.0,156.555,12.0,9.0,14.0,15.0,6.0,8.0,12.0,13.0,14.0,12.0,22.0,17.0,36.0,43.0,36.0,34.0,51.0,20.0,51.0,8.0,13.0,11.0,7.0,18.0,15.0,17.0,19.406682,11.0,9.0,49.0,48.0,45.0,38.0,52.0,47.0,223000.0,21.000000,2022.0,-108.0,0,0,1,1,3.0,0


In [31]:
# encoder = ce.BinaryEncoder(cols=['Position'],return_df=True)
# df_multivariate_imputed = encoder.fit_transform(df_multivariate_imputed)
# df_multivariate_imputed

In [32]:
# encoder = ce.BinaryEncoder(cols=['Best Position'],return_df=True)
# df_multivariate_imputed = encoder.fit_transform(df_multivariate_imputed)
# df_multivariate_imputed

In [33]:
df_multivariate_imputed = df_multivariate_imputed.rename(columns=dict(zip(df_multivariate_imputed.columns, numerical_features)))
df_multivariate_imputed

,Age,Overall,Potential,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Jersey Number,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Overall Rating,Release Clause,DefensiveAwareness,year,Days_of_contract,Body Type_0,Body Type_1,Body Type_2,Body Type_3,Work_Rate_transformed,Real_Face_transformed
0,26.0,88.0,89.0,250000.0,2341.0,1.0,3.0,3.0,4.0,18.0,70.0,152.145,87.0,83.0,64.0,91.0,87.0,83.0,87.0,87.0,88.0,87.0,77.0,73.0,80.0,91.0,79.0,89.0,73.0,91.0,70.0,89.0,78.0,66.0,87.0,90.0,91.0,87.0,66.363477,73.0,65.0,12.0,14.0,15.0,8.0,14.0,88.0,206900.0,72.000000,2022.0,1798.0,0,0,0,1,9.0,1
1,26.0,87.0,88.0,140000.0,2314.0,1.0,4.0,4.0,3.0,8.0,74.0,180.810,75.0,82.0,86.0,86.0,69.0,84.0,76.0,75.0,84.0,87.0,78.0,83.0,76.0,88.0,71.0,85.0,79.0,88.0,88.0,86.0,81.0,86.0,85.0,84.0,60.0,82.0,72.193359,85.0,77.0,13.0,8.0,15.0,11.0,9.0,87.0,160400.0,74.000000,2022.0,2741.0,0,0,0,1,8.0,1
2,34.0,88.0,88.0,135000.0,2307.0,1.0,5.0,4.0,3.0,9.0,71.0,183.015,80.0,93.0,84.0,83.0,90.0,83.0,86.0,82.0,77.0,86.0,76.0,69.0,75.0,92.0,78.0,89.0,69.0,78.0,85.0,88.0,87.0,41.0,91.0,84.0,83.0,87.0,41.260758,45.0,38.0,27.0,25.0,31.0,33.0,37.0,88.0,91200.0,42.000000,2022.0,463.0,0,0,0,1,8.0,1
3,30.0,91.0,91.0,350000.0,2304.0,1.0,4.0,5.0,4.0,17.0,71.0,154.350,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93.0,91.0,76.0,76.0,79.0,91.0,78.0,91.0,63.0,89.0,74.0,91.0,76.0,66.0,88.0,94.0,83.0,89.0,61.788810,65.0,53.0,15.0,13.0,5.0,10.0,13.0,91.0,232200.0,68.000000,2022.0,3412.0,0,0,0,1,9.0,1
4,29.0,84.0,84.0,45000.0,2292.0,0.0,2.0,3.0,4.0,19.0,67.0,152.145,87.0,66.0,58.0,82.0,68.0,87.0,88.0,75.0,78.0,88.0,77.0,76.0,83.0,83.0,90.0,82.0,63.0,90.0,80.0,81.0,84.0,79.0,81.0,82.0,76.0,87.0,74.381081,84.0,82.0,8.0,14.0,13.0,13.0,14.0,84.0,77700.0,80.000000,2022.0,1204.0,0,0,1,0,9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16411,20.0,58.0,69.0,950.0,802.0,1.0,1.0,2.0,1.0,30.0,76.0,187.425,8.0,9.0,11.0,23.0,7.0,11.0,11.0,11.0,17.0,14.0,24.0,25.0,27.0,42.0,20.0,42.0,22.0,21.0,67.0,8.0,21.0,4.0,5.0,27.0,7.0,29.0,10.630345,12.0,10.0,59.0,60.0,56.0,55.0,61.0,58.0,656000.0,5.000000,2022.0,915.0,0,1,1,0,3.0,0
16412,17.0,53.0,70.0,1000.0,782.0,1.0,1.0,2.0,1.0,31.0,74.0,176.400,12.0,5.0,13.0,18.0,7.0,12.0,11.0,13.0,19.0,10.0,15.0,21.0,34.0,30.0,31.0,16.0,51.0,26.0,57.0,6.0,16.0,9.0,5.0,29.0,18.0,37.0,5.000000,11.0,12.0,55.0,54.0,52.0,50.0,59.0,52.0,238000.0,-2.823051,2022.0,914.0,0,1,1,0,3.0,0
16413,20.0,53.0,63.0,550.0,778.0,1.0,1.0,2.0,1.0,1.0,76.0,189.630,13.0,7.0,10.0,17.0,7.0,6.0,11.0,13.0,28.0,17.0,16.0,22.0,30.0,38.0,24.0,40.0,35.0,24.0,32.0,6.0,18.0,13.0,8.0,33.0,12.0,29.0,12.165843,13.0,12.0,57.0,52.0,53.0,48.0,58.0,53.0,279000.0,5.000000,2022.0,2349.0,0,1,1,0,3.0,0
16414,19.0,47.0,60.0,700.0,770.0,1.0,1.0,2.0,1.0,31.0,70.0,156.555,12.0,9.0,14.0,15.0,6.0,8.0,12.0,13.0,14.0,12.0,22.0,17.0,36.0,43.0,36.0,34.0,51.0,20.0,51.0,8.0,13.0,11.0,7.0,18.0,15.0,17.0,19.406682,11.0,9.0,49.0,48.0,45.0,38.0,52.0,47.0,223000.0,21.000000,2022.0,-108.0,0,0,1,1,3.0,0


In [34]:
features_to_be_normalised = ['Age', 'Wage', 'Weak Foot', 'Skill Moves', 'Jersey Number', 'Weight', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']
df_multivariate_imputed[features_to_be_normalised] = log_normal_transform(df_multivariate_imputed[features_to_be_normalised])
for column in df_multivariate_imputed.columns:
    num_outliers = count_outliers_zscore(df_multivariate_imputed[column])
    print(f"Number of outliers in {column}: {num_outliers}")

Number of outliers in Age: 1
Number of outliers in Overall: 79
Number of outliers in Potential: 55
Number of outliers in Wage: 21
Number of outliers in Special: 117
Number of outliers in Preferred Foot: 0
Number of outliers in International Reputation: 533
Number of outliers in Weak Foot: 97
Number of outliers in Skill Moves: 0
Number of outliers in Jersey Number: 0
Number of outliers in Height: 12
Number of outliers in Weight: 21
Number of outliers in Crossing: 0
Number of outliers in Finishing: 0
Number of outliers in HeadingAccuracy: 0
Number of outliers in ShortPassing: 228
Number of outliers in Volleys: 0
Number of outliers in Dribbling: 0
Number of outliers in Curve: 0
Number of outliers in FKAccuracy: 0
Number of outliers in LongPassing: 24
Number of outliers in BallControl: 258
Number of outliers in Acceleration: 93
Number of outliers in SprintSpeed: 89
Number of outliers in Agility: 17
Number of outliers in Reactions: 73
Number of outliers in Balance: 37
Number of outliers in 

In [35]:
df_multivariate_imputed.to_csv('/content/drive/MyDrive/Mltiverse/MLtiVerse (1)/test_transformed.csv', index=False)

In [36]:
features = ['Club', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Volleys', 'Curve', 'Agility', 'Balance', 'Jumping', 'Interceptions', 'Positioning', 'Vision', 'SlidingTackle']
df_null = df[df[features].isnull().any(axis=1)]

In [37]:
df_null

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Loaned From,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness,year,Days_of_contract
88,187457,19 C. Noboa,33,https://cdn.sofifa.com/players/187/457/19_60.png,Ecuador,https://cdn.sofifa.com/flags/ec.png,79,79,NaN,https://cdn.sofifa.com/flags/ec.png,0.0,2168,1,2.0,3.0,4.0,High/ High,Normal (170-185),No,"<span class=""pos pos28"">SUB",6.0,NaN,72,165.375,74.0,65.0,72.0,78.0,66.0,77.0,76.0,81.0,80.0,80.0,73.0,73.0,77.0,75.0,66.0,75.0,79.0,90.0,71.0,79.0,68.0,78.0,72.0,78.0,80.0,72.0,75.0,75.0,75.0,15.0,8.0,11.0,15.0,11.0,CM,78.0,2200000.0,NaN,2022,NaN
137,203535,20 T. Vilhena,24,https://cdn.sofifa.com/players/203/535/20_60.png,Netherlands,https://cdn.sofifa.com/flags/nl.png,78,82,NaN,https://cdn.sofifa.com/flags/nl.png,0.0,2139,0,2.0,4.0,3.0,High/ High,Stocky (170-185),No,"<span class=""pos pos28"">SUB",18.0,NaN,68,154.350,76.0,71.0,69.0,78.0,68.0,77.0,74.0,69.0,77.0,80.0,77.0,73.0,76.0,77.0,83.0,80.0,80.0,85.0,75.0,76.0,81.0,75.0,70.0,73.0,53.0,80.0,NaN,72.0,68.0,12.0,8.0,16.0,12.0,12.0,CM,77.0,2200000.0,66.0,2022,NaN
169,146562,21 Santi Cazorla,35,https://cdn.sofifa.com/players/146/562/21_60.png,Spain,https://cdn.sofifa.com/flags/es.png,82,82,NaN,https://cdn.sofifa.com/flags/es.png,0.0,2129,1,3.0,5.0,4.0,High/ Medium,Stocky (170-),Yes,"<span class=""pos pos28"">SUB",20.0,NaN,66,143.325,85.0,78.0,60.0,86.0,72.0,85.0,87.0,78.0,84.0,87.0,64.0,61.0,76.0,81.0,90.0,77.0,50.0,60.0,59.0,80.0,67.0,73.0,80.0,86.0,90.0,88.0,NaN,65.0,58.0,6.0,9.0,5.0,7.0,15.0,CM,82.0,2200000.0,68.0,2022,NaN
342,197242,G. Krychowiak,31,https://cdn.sofifa.com/players/197/242/22_60.png,Poland,https://cdn.sofifa.com/flags/pl.png,80,80,NaN,https://cdn.sofifa.com/flags/pl.png,0.0,2081,1,3.0,3.0,3.0,High/ High,Normal (185+),No,"<span class=""pos pos10"">CDM",10.0,NaN,73,185.220,63.0,72.0,75.0,78.0,75.0,70.0,54.0,62.0,81.0,79.0,67.0,69.0,59.0,80.0,53.0,82.0,74.0,78.0,83.0,78.0,80.0,77.0,72.0,69.0,78.0,80.0,NaN,76.0,72.0,15.0,14.0,8.0,6.0,7.0,CDM,80.0,2200000.0,75.0,2022,NaN
401,245324,21 S. Luna,32,https://cdn.sofifa.com/players/245/324/21_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,78,78,NaN,https://cdn.sofifa.com/flags/uy.png,0.0,2067,1,1.0,3.0,4.0,Medium/ Medium,Normal (170-185),No,"<span class=""pos pos28"">SUB",19.0,NaN,69,165.375,76.0,69.0,60.0,76.0,71.0,76.0,76.0,68.0,59.0,75.0,84.0,76.0,79.0,80.0,70.0,72.0,61.0,88.0,68.0,74.0,73.0,72.0,76.0,64.0,64.0,80.0,NaN,75.0,77.0,7.0,8.0,8.0,8.0,7.0,RB,78.0,2200000.0,70.0,2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16527,257550,I. Al Kaabi,28,https://cdn.sofifa.com/players/257/550/22_60.png,United Arab Emirates,https://cdn.sofifa.com/flags/ae.png,59,62,NaN,https://cdn.sofifa.com/flags/ae.png,0.0,910,1,1.0,3.0,1.0,Medium/ Medium,Normal (170-185),No,NaN,NaN,NaN,72,180.810,13.0,9.0,10.0,24.0,6.0,13.0,10.0,10.0,17.0,16.0,34.0,33.0,23.0,54.0,47.0,44.0,60.0,26.0,46.0,8.0,24.0,10.0,5.0,30.0,19.0,31.0,NaN,10.0,11.0,59.0,55.0,58.0,58.0,60.0,GK,59.0,2200000.0,8.0,2022,NaN
16540,247066,21 H. Al Mansoury,24,https://cdn.sofifa.com/players/247/066/21_60.png,United Arab Emirates,https://cdn.sofifa.com/flags/ae.png,58,63,NaN,https://cdn.sofifa.com/flags/ae.png,0.0,904,1,1.0,2.0,1.0,Medium/ Medium,Normal (170-185),No,NaN,NaN,NaN,70

In [38]:
df_null["Value"] = 0

<ipython-input-38-09a9f44ee592>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_null["Value"] = 0


In [39]:
df_null

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Loaned From,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness,year,Days_of_contract,Value
88,187457,19 C. Noboa,33,https://cdn.sofifa.com/players/187/457/19_60.png,Ecuador,https://cdn.sofifa.com/flags/ec.png,79,79,NaN,https://cdn.sofifa.com/flags/ec.png,0.0,2168,1,2.0,3.0,4.0,High/ High,Normal (170-185),No,"<span class=""pos pos28"">SUB",6.0,NaN,72,165.375,74.0,65.0,72.0,78.0,66.0,77.0,76.0,81.0,80.0,80.0,73.0,73.0,77.0,75.0,66.0,75.0,79.0,90.0,71.0,79.0,68.0,78.0,72.0,78.0,80.0,72.0,75.0,75.0,75.0,15.0,8.0,11.0,15.0,11.0,CM,78.0,2200000.0,NaN,2022,NaN,0
137,203535,20 T. Vilhena,24,https://cdn.sofifa.com/players/203/535/20_60.png,Netherlands,https://cdn.sofifa.com/flags/nl.png,78,82,NaN,https://cdn.sofifa.com/flags/nl.png,0.0,2139,0,2.0,4.0,3.0,High/ High,Stocky (170-185),No,"<span class=""pos pos28"">SUB",18.0,NaN,68,154.350,76.0,71.0,69.0,78.0,68.0,77.0,74.0,69.0,77.0,80.0,77.0,73.0,76.0,77.0,83.0,80.0,80.0,85.0,75.0,76.0,81.0,75.0,70.0,73.0,53.0,80.0,NaN,72.0,68.0,12.0,8.0,16.0,12.0,12.0,CM,77.0,2200000.0,66.0,2022,NaN,0
169,146562,21 Santi Cazorla,35,https://cdn.sofifa.com/players/146/562/21_60.png,Spain,https://cdn.sofifa.com/flags/es.png,82,82,NaN,https://cdn.sofifa.com/flags/es.png,0.0,2129,1,3.0,5.0,4.0,High/ Medium,Stocky (170-),Yes,"<span class=""pos pos28"">SUB",20.0,NaN,66,143.325,85.0,78.0,60.0,86.0,72.0,85.0,87.0,78.0,84.0,87.0,64.0,61.0,76.0,81.0,90.0,77.0,50.0,60.0,59.0,80.0,67.0,73.0,80.0,86.0,90.0,88.0,NaN,65.0,58.0,6.0,9.0,5.0,7.0,15.0,CM,82.0,2200000.0,68.0,2022,NaN,0
342,197242,G. Krychowiak,31,https://cdn.sofifa.com/players/197/242/22_60.png,Poland,https://cdn.sofifa.com/flags/pl.png,80,80,NaN,https://cdn.sofifa.com/flags/pl.png,0.0,2081,1,3.0,3.0,3.0,High/ High,Normal (185+),No,"<span class=""pos pos10"">CDM",10.0,NaN,73,185.220,63.0,72.0,75.0,78.0,75.0,70.0,54.0,62.0,81.0,79.0,67.0,69.0,59.0,80.0,53.0,82.0,74.0,78.0,83.0,78.0,80.0,77.0,72.0,69.0,78.0,80.0,NaN,76.0,72.0,15.0,14.0,8.0,6.0,7.0,CDM,80.0,2200000.0,75.0,2022,NaN,0
401,245324,21 S. Luna,32,https://cdn.sofifa.com/players/245/324/21_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,78,78,NaN,https://cdn.sofifa.com/flags/uy.png,0.0,2067,1,1.0,3.0,4.0,Medium/ Medium,Normal (170-185),No,"<span class=""pos pos28"">SUB",19.0,NaN,69,165.375,76.0,69.0,60.0,76.0,71.0,76.0,76.0,68.0,59.0,75.0,84.0,76.0,79.0,80.0,70.0,72.0,61.0,88.0,68.0,74.0,73.0,72.0,76.0,64.0,64.0,80.0,NaN,75.0,77.0,7.0,8.0,8.0,8.0,7.0,RB,78.0,2200000.0,70.0,2022,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16527,257550,I. Al Kaabi,28,https://cdn.sofifa.com/players/257/550/22_60.png,United Arab Emirates,https://cdn.sofifa.com/flags/ae.png,59,62,NaN,https://cdn.sofifa.com/flags/ae.png,0.0,910,1,1.0,3.0,1.0,Medium/ Medium,Normal (170-185),No,NaN,NaN,NaN,72,180.810,13.0,9.0,10.0,24.0,6.0,13.0,10.0,10.0,17.0,16.0,34.0,33.0,23.0,54.0,47.0,44.0,60.0,26.0,46.0,8.0,24.0,10.0,5.0,30.0,19.0,31.0,NaN,10.0,11.0,59.0,55.0,58.0,58.0,60.0,GK,59.0,2200000.0,8.0,2022,NaN,0
16540,247066,21 H. Al Mansoury,24,https://cdn.sofifa.com/players/247/066/21_60.png,United Arab Emirates,https://cdn.sofifa.com/flags/ae.png,58,63,NaN,https://cdn.sofifa.com/flags/ae.png,0.0,904,1,1.0,2.0,1.0,Medium/ Medium,Normal (170-

In [40]:
df_null_result = df_null[["Photo", "Value"]]

In [41]:
df_null_result

,Photo,Value
88,https://cdn.sofifa.com/players/187/457/19_60.png,0
137,https://cdn.sofifa.com/players/203/535/20_60.png,0
169,https://cdn.sofifa.com/players/146/562/21_60.png,0
342,https://cdn.sofifa.com/players/197/242/22_60.png,0
401,https://cdn.sofifa.com/players/245/324/21_60.png,0
...,...,...
16527,https://cdn.sofifa.com/players/257/550/22_60.png,0
16540,https://cdn.sofifa.com/players/247/066/21_60.png,0
16572,https://cdn.sofifa.com/players/019/334/10_60.png,0
16585,https://cdn.sofifa.com/players/252/359/21_60.png,0


In [42]:
df_null_result.to_csv("/content/drive/MyDrive/Mltiverse/MLtiVerse (1)/df_null_result.csv", index = False)